# Custom DataLoader

In [90]:
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Union, Optional

import torch
from torch.utils.data import Dataset
import pytorch_lightning as pl

data_dir = Path("/cats/datastore/data")

In [40]:
ds = pickle.load((data_dir / "kenya.pkl").open("rb"))
ds

<xarray.Dataset>
Dimensions:     (lat: 45, lon: 35, time: 205)
Coordinates:
  * time        (time) datetime64[ns] 2001-12-31 2002-01-31 ... 2018-12-31
  * lat         (lat) float32 6.0 5.75 5.5 5.25 5.0 ... -4.25 -4.5 -4.75 -5.0
  * lon         (lon) float32 33.75 34.0 34.25 34.5 ... 41.5 41.75 42.0 42.25
Data variables:
    VCI3M       (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    boku_VCI    (time, lat, lon) float64 43.72 31.13 85.88 20.18 ... nan nan nan
    modis_ndvi  (time, lat, lon) float64 0.4688 0.4032 0.624 ... nan nan nan
    VCI         (time, lat, lon) float64 39.64 37.79 89.79 53.54 ... nan nan nan
    precip      (time, lat, lon) float64 13.66 18.42 24.78 47.38 ... nan nan nan
    E           (time, lat, lon) float64 52.86 64.59 64.59 75.49 ... nan nan nan
    Eb          (time, lat, lon) float64 2.3 2.311 2.311 2.409 ... nan nan nan
    SMroot      (time, lat, lon) float64 0.2993 0.3273 0.3273 ... nan nan nan
    SMsurf      (time, lat, lon) float64 0.2837 0.309 0.309 ... nan nan nan
    p84.162     (time, lat, lon) float64 1.121e-05 3.69e-05 ... nan nan
    pev         (time, lat, lon) float64 -0.007123 -0.007165 ... nan nan
    sp          (time, lat, lon) float64 9.591e+04 9.604e+04 ... nan nan
    t2m         (time, lat, lon) float64 303.0 303.8 302.2 299.3 ... nan nan nan
    tp          (time, lat, lon) float64 0.0004198 0.0003483 ... nan nan

In [96]:
target_var = "boku_VCI"
pixel_vars: List[str] = ["lat", "lon"]
in_lagged_variables = ["boku_VCI"]
dynamic_inputs = []

df = create_pixel_dataframe(ds, pixel_vars)

lagged_variables = []
for var in in_lagged_variables:
    df, var_name = _add_lagged_feature(df, var, lag=1)
    lagged_variables.append(var_name)

keep_cols = ["pixel"] + dynamic_inputs + [target_var] + lagged_variables
df = df[keep_cols]
df.head()

,pixel,boku_VCI,boku_VCI_shift1
time,,,
2001-12-31,6.0_33.75,43.718593,NaN
2002-01-31,6.0_33.75,2.843602,43.718593
2002-02-28,6.0_33.75,34.337349,2.843602
2002-03-31,6.0_33.75,50.000000,34.337349
2002-04-30,6.0_33.75,54.945055,50.000000


In [55]:
def compute_scaler(df: pd.DataFrame) -> Dict[str, pd.Series]:
    scaler = {}
    scaler["attribute_means"] = df.mean()
    scaler["attribute_stds"] = df.std()
    return scaler

In [136]:
train_period_start = "2001-01-01"
train_period_end = "2015-12-01"
test_period_start = "2016-01-01"
test_period_end = "2016-01-01"
# df.loc[test_period_start: test_period_end]

In [166]:
# pd.infer_freq([pd.to_datetime(d) for d in df.index])
df.index = [pd.Timestamp(dt) for dt in df.index]
dt = pd.Timestamp(train_period_start)
# df.index[df.index.get_loc(dt, method='nearest')]
# df.index.get_loc(dt, method='nearest')
# df.loc[dt]
null_pixels = _get_null_pixels(df, "boku_VCI")
df = df.loc[~np.isin(df["pixel"], null_pixels)]

,pixel,boku_VCI,boku_VCI_shift1
2001-12-31,6.0_33.75,43.718593,NaN
2002-01-31,6.0_33.75,2.843602,43.718593
2002-02-28,6.0_33.75,34.337349,2.843602
2002-03-31,6.0_33.75,50.000000,34.337349
2002-04-30,6.0_33.75,54.945055,50.000000
...,...,...,...
2018-08-31,-5.0_42.25,NaN,NaN
2018-09-30,-5.0_42.25,NaN,NaN
2018-10-31,-5.0_42.25,NaN,NaN
2018-11-30,-5.0_42.25,NaN,NaN


In [233]:
# df.index[df.index.get_loc(dt, method='nearest')]
dt = pd.to_datetime("2001-12-31")
# df.index.get_loc(dt, method='nearest')
df.loc[dt]

,pixel,boku_VCI,boku_VCI_shift1
2001-12-31,6.0_33.75,43.718593,NaN
2001-12-31,6.0_34.0,31.132075,34.170854
2001-12-31,6.0_34.25,85.882353,12.264151
2001-12-31,6.0_34.5,20.183486,51.764706
2001-12-31,6.0_34.75,58.447489,17.431193
...,...,...,...
2001-12-31,-5.0_38.25,0.282486,74.461538
2001-12-31,-5.0_38.5,23.036649,70.338983
2001-12-31,-5.0_38.75,31.147541,70.680628
2001-12-31,-5.0_39.0,0.000000,68.032787


In [12]:
from torch.utils.data import random_split, DataLoader

In [127]:
import pickle

scale = pickle.load(Path("/cats/datastore/data/runs/ensemble_EALSTM/ealstm_ensemble1_nse_1998_2008_2810_162619/train_data/train_data_scaler.p").open("rb"))
scale.keys()

dict_keys(['attribute_means', 'attribute_stds', 'xarray_feature_scale', 'xarray_feature_center'])

In [201]:
def _get_null_pixels(df: pd.DataFrame, target_var: str) -> pd.Series:
    return df["pixel"].loc[df[target_var].isnull()].unique()


In [315]:
def _stack_xarray(ds:xr.Dataset, spatial_coords: List[str]) -> xr.Dataset:
    # stack values
    stacked = ds.stack(sample=spatial_coords)
    samples = stacked.sample
    pixel_strs = [f"{ll[0]}_{ll[-1]}" for ll in samples.values]
    stacked["sample"] = pixel_strs

    samples = samples.to_dataset(name="pixel")
    samples = xr.DataArray(pixel_strs, dims=["sample"], coords={"sample": samples.sample})
    return stacked, samples

In [319]:
stacked_ds, samples = _stack_xarray(ds, pixel_vars)
stacked_ds

<xarray.Dataset>
Dimensions:          (sample: 1575, time: 205)
Coordinates:
  * time             (time) datetime64[ns] 2001-12-31 2002-01-31 ... 2018-12-31
  * sample           (sample) <U11 '6.0_33.75' '6.0_34.0' ... '-5.0_42.25'
Data variables:
    VCI3M            (time, sample) float64 nan nan nan nan ... nan nan nan nan
    boku_VCI         (time, sample) float64 43.72 31.13 85.88 ... nan nan nan
    modis_ndvi       (time, sample) float64 0.4688 0.4032 0.624 ... nan nan nan
    VCI              (time, sample) float64 39.64 37.79 89.79 ... nan nan nan
    precip           (time, sample) float64 13.66 18.42 24.78 ... nan nan nan
    E                (time, sample) float64 52.86 64.59 64.59 ... nan nan nan
    Eb               (time, sample) float64 2.3 2.311 2.311 ... nan nan nan
    SMroot           (time, sample) float64 0.2993 0.3273 0.3273 ... nan nan nan
    SMsurf           (time, sample) float64 0.2837 0.309 0.309 ... nan nan nan
    p84.162          (time, sample) float64 1.121e-05 3.69e-05 ... nan nan
    pev              (time, sample) float64 -0.007123 -0.007165 ... nan nan
    sp               (time, sample) float64 9.591e+04 9.604e+04 ... nan nan
    t2m              (time, sample) float64 303.0 303.8 302.2 ... nan nan nan
    tp               (time, sample) float64 0.0004198 0.0003483 ... nan nan
    boku_VCI_shift1  (time, sample) float64 nan nan nan nan ... nan nan nan nan

In [353]:
isnull = stacked_ds[target_var].isnull().mean(dim=["time"]) == 1
isnull = isnull.to_dataframe().reset_index()
pixel_var = [c for c in stacked_ds.coords if c!="time"][0]
null_pixels = isnull[pixel_var].loc[isnull[target_var].astype(bool)]


# stacked_ds
stacked_ds = stacked_ds.sel(sample = ~np.isin(stacked_ds.sample, null_pixels.values))

In [356]:
stacked_ds

<xarray.Dataset>
Dimensions:          (sample: 1433, time: 205)
Coordinates:
  * time             (time) datetime64[ns] 2001-12-31 2002-01-31 ... 2018-12-31
  * sample           (sample) <U11 '6.0_33.75' '6.0_34.0' ... '-5.0_39.75'
Data variables:
    VCI3M            (time, sample) float64 nan nan nan nan ... nan nan nan nan
    boku_VCI         (time, sample) float64 43.72 31.13 85.88 ... 77.48 2.247
    modis_ndvi       (time, sample) float64 0.4688 0.4032 0.624 ... nan nan nan
    VCI              (time, sample) float64 39.64 37.79 89.79 ... nan nan nan
    precip           (time, sample) float64 13.66 18.42 24.78 ... nan nan nan
    E                (time, sample) float64 52.86 64.59 64.59 ... nan nan nan
    Eb               (time, sample) float64 2.3 2.311 2.311 ... nan nan nan
    SMroot           (time, sample) float64 0.2993 0.3273 0.3273 ... nan nan nan
    SMsurf           (time, sample) float64 0.2837 0.309 0.309 ... nan nan nan
    p84.162          (time, sample) float64 1.121e-05 3.69e-05 ... nan nan
    pev              (time, sample) float64 -0.007123 -0.007165 ... nan nan
    sp               (time, sample) float64 9.591e+04 9.604e+04 ... nan nan
    t2m              (time, sample) float64 303.0 303.8 302.2 ... nan nan nan
    tp               (time, sample) float64 0.0004198 0.0003483 ... nan nan
    boku_VCI_shift1  (time, sample) float64 nan nan nan ... 59.65 65.54 23.66

In [477]:
sample_coordinates = ds["sample"].values.tolist()
dynamic_inputs = ["precip"]

def _check_no_missing_times_in_time_series(df):
    inf_freq = pd.infer_freq(df.index)
    assert list(pd.date_range(start=min_timestamp, end=max_timestamp, freq=inf_freq).difference(df.index)) == [], f"Missing data"


# store data
# create a lookup object from INDEX to PIXEL str
lookup = []
x_d, x_s, y = {}, {}, {}
for sample in tqdm(sample_coordinates, file=sys.stdout):    
    # get raw dataframe
    keep_cols = [target_var] + dynamic_inputs + lagged_variables
    df_native = stacked_ds[keep_cols].sel({"sample": sample}).to_dataframe()
    # ensure that sorted by time
    df_native = df_native.sort_index()
    # ensure that every expected timestep is in the dataframe
    _check_no_missing_times_in_time_series(df_native)
    
    y[sample] = df_native[target_var].to_numpy().reshape(-1, 1)
    x_d[sample] = df_native[dynamic_inputs + lagged_variables].to_numpy()

    valid_samples = np.argwhere(flag == 1)
    for f in valid_samples:
        # store pointer to pixel and the sample's index
        lookup.append((basin, [frequency_maps[freq][int(f)] for freq in self.frequencies]))
    

self.lookup_table = {i: elem for i, elem in enumerate(lookup)}
self.num_samples = len(self.lookup_table)

  0%|          | 0/1433 [00:00<?, ?it/s]


In [480]:
def __getitem__(self, item: int) -> Dict[str, torch.Tensor]:
    basin, 

,boku_VCI,precip,boku_VCI_shift1,sample
time,,,,
2001-12-31,43.718593,13.663948,NaN,6.0_33.75
2002-01-31,2.843602,4.231649,43.718593,6.0_33.75
2002-02-28,34.337349,5.615317,2.843602,6.0_33.75
2002-03-31,50.000000,71.845627,34.337349,6.0_33.75
2002-04-30,54.945055,43.818996,50.000000,6.0_33.75
...,...,...,...,...
2018-08-31,85.382060,145.350433,100.000000,6.0_33.75
2018-09-30,49.115044,115.616341,85.382060,6.0_33.75
2018-10-31,54.435484,81.578751,49.115044,6.0_33.75


In [472]:
min_timestamp = df_native.index.min()
max_timestamp = df_native.index.max()

df_native.index.get_loc(min_timestamp, method='nearest') + 3

# df.index[df.index.get_loc(min_timestamp, method='nearest')]

3

In [460]:
    
_check_no_missing_times_in_time_series(df_native)

In [433]:
inf_freq = pd.infer_freq(df_native.index)
if inf_freq == "M":
    freq = "D"
    value = 30 * seq_len
elif inf_freq == "Y":
    freq = "D"
    value = 366 * seq_len
elif inf_freq == "W":
    freq = "D"
    value = 7 * seq_len
else:
    freq = inf_freq
    value = seq_len
    
pd.Timedelta(value=value, unit="D")

Timedelta('90 days 00:00:00')

In [464]:
# ?pd.Timedelta

In [417]:
stacked_ds[keep_cols]

<xarray.Dataset>
Dimensions:          (sample: 1433, time: 205)
Coordinates:
  * time             (time) datetime64[ns] 2001-12-31 2002-01-31 ... 2018-12-31
  * sample           (sample) <U11 '6.0_33.75' '6.0_34.0' ... '-5.0_39.75'
Data variables:
    boku_VCI         (time, sample) float64 43.72 31.13 85.88 ... 77.48 2.247
    precip           (time, sample) float64 13.66 18.42 24.78 ... nan nan nan
    boku_VCI_shift1  (time, sample) float64 nan nan nan ... 59.65 65.54 23.66

In [415]:
df_native.head()


,boku_VCI,precip,boku_VCI_shift1,sample
time,,,,
2001-12-31,43.718593,13.663948,NaN,6.0_33.75
2002-01-31,2.843602,4.231649,43.718593,6.0_33.75
2002-02-28,34.337349,5.615317,2.843602,6.0_33.75
2002-03-31,50.000000,71.845627,34.337349,6.0_33.75
2002-04-30,54.945055,43.818996,50.000000,6.0_33.75
...,...,...,...,...
2018-08-31,85.382060,145.350433,100.000000,6.0_33.75
2018-09-30,49.115044,115.616341,85.382060,6.0_33.75
2018-10-31,54.435484,81.578751,49.115044,6.0_33.75


In [413]:
seq_len = 3
df_time_min = df_native.index.min()

Timestamp('2001-12-31 00:00:00')

In [395]:
df_nat

,boku_VCI,boku_VCI_shift1,sample
time,,,
2001-12-31,43.718593,NaN,6.0_33.75
2002-01-31,2.843602,43.718593,6.0_33.75
2002-02-28,34.337349,2.843602,6.0_33.75
2002-03-31,50.000000,34.337349,6.0_33.75
2002-04-30,54.945055,50.000000,6.0_33.75
...,...,...,...
2018-08-31,85.382060,100.000000,6.0_33.75
2018-09-30,49.115044,85.382060,6.0_33.75
2018-10-31,54.435484,49.115044,6.0_33.75


In [408]:
df_native.loc[:, [target_var] + dynamic_inputs + lagged_variables]

,boku_VCI,precip,boku_VCI_shift1
time,,,
2001-12-31,43.718593,13.663948,NaN
2002-01-31,2.843602,4.231649,43.718593
2002-02-28,34.337349,5.615317,2.843602
2002-03-31,50.000000,71.845627,34.337349
2002-04-30,54.945055,43.818996,50.000000
...,...,...,...
2018-08-31,85.382060,145.350433,100.000000
2018-09-30,49.115044,115.616341,85.382060
2018-10-31,54.435484,81.578751,49.115044


In [409]:
df

,pixel,boku_VCI,boku_VCI_shift1
2001-12-31,6.0_33.75,43.718593,NaN
2002-01-31,6.0_33.75,2.843602,43.718593
2002-02-28,6.0_33.75,34.337349,2.843602
2002-03-31,6.0_33.75,50.000000,34.337349
2002-04-30,6.0_33.75,54.945055,50.000000
...,...,...,...
2018-08-31,-5.0_39.75,71.875000,91.836735
2018-09-30,-5.0_39.75,70.000000,71.875000
2018-10-31,-5.0_39.75,52.054795,70.000000
2018-11-30,-5.0_39.75,23.655914,52.054795


['boku_VCI', 'boku_VCI_shift1']

In [407]:
from torch.utils.data import random_split, DataLoader
import sys
from tqdm import tqdm


def _join_latlon_cols(df: pd.DataFrame, join_str: str = "_") -> pd.Series:
    points = df.astype({"lat": "str", "lon": "str"})
    return points['lat'] + join_str +  points['lon']

def _latlon_as_strings(ds, join_str: str = "_"):
    points = ds[["lat", "lon"]].to_dataframe().reset_index()
    return _join_latlon_cols(points, join_str=join_str)

def create_pixel_dataframe(ds: xr.Dataset, pixel_vars: List[str]) -> pd.DataFrame:
    df = ds.to_dataframe().reset_index(pixel_vars)
    df["pixel"] = _join_latlon_cols(df)
    return df

def _add_lagged_feature(ds: xr.Dataset, variable: str, lag: int = 1) -> Tuple[xr.Dataset, str]:
    var_name = f"{variable}_shift{lag}"
    ds[var_name] = ds[variable].shift(time=1)
    return ds, var_name

def create_pixel_id_encoding():
    id_to_int = {b: i for i, b in enumerate(np.random.permutation(self.pixels))}

    
def _get_null_pixels(df: pd.DataFrame, target_var: str) -> pd.Series:
    return df["pixel"].loc[df[target_var].isnull()].unique()


def get_all_null_pixels_from_dataset(ds, target_var: str) -> pd.Series:
    assert len(ds.coords) == 2, f"Expect coords = [time sample], got: {[c for c in ds.coords]}"
    isnull = ds[target_var].isnull().mean(dim=["time"]) == 1
    isnull = isnull.to_dataframe().reset_index()    
    pixel_var = [c for c in ds.coords if c!="time"][0]
    null_pixels = isnull[pixel_var].loc[isnull[target_var].astype(bool)]
    
    return null_pixels


def _stack_xarray(ds: xr.Dataset, spatial_coords: List[str]) -> xr.Dataset:
    # stack values
    stacked = ds.stack({"sample": spatial_coords})
    samples = stacked["sample"]
    pixel_strs = [f"{ll[0]}_{ll[-1]}" for ll in samples.values]
    stacked["sample"] = pixel_strs

    samples = samples.to_dataset(name="pixel")
    samples = xr.DataArray(pixel_strs, dims=["sample"], coords={"sample": samples.sample})
    return stacked, samples


class SpatioTemporalDataset(Dataset):
    """
    Tasks:
    -----
    • normalise input and output data
    • split into X, y variables (per-pixel)
    • split X into: static, dynamic data
    • convert xr.Dataset -> torch.Tensor
    • keep track of the metadata (time, lat, lon) [Ask Sharan]
    • create train, test, validation
    • Remove missing data
    • Create lagged target variable as input
    
    To implement:
    ------------
    prepare_data (how to download(), tokenize, etc…)
    setup (how to split, etc…)
    train_dataloader
    val_dataloader(s)
    test_dataloader(s)
    """    
    def __init__(
        self, 
        ds: xr.Dataset, 
        target_var: str,
        seq_len: int,
        forecast_horizon: int = 1,
        static_ds: Optional[xr.Dataset] = None,
        dynamic_inputs: List[str] = [],
        static_inputs: List[str] = [],
        batch_size: int = 32, 
        pixel_vars: List[str] = ["lat", "lon"],
        use_pixel_id_encoding: bool = False,
        lagged_variables: List[str] = [],
        lag: int = 1,
    ):
        super().__init__()
        self.batch_size = batch_size
        self.pixel_vars = pixel_vars
        self.lag = lag
        self.seq_len
        self.forecast_horizon = forecast_horizon
        
        # get X, y variables
        self.target_var = target_var
        self.dynamic_inputs = dynamic_inputs
        self.lagged_variables = lagged_variables 
        
        # run checks on input arguments
        self._init_checks(ds)
        
        self.ds = ds
        self.scaler: Dict[str, pd.Series] = {}
            
    def _create_lookup_table(self, ds: xr.Dataset):
        samples_without_data = []
        sample_coordinates = ds["sample"].values.tolist()
        
        for sample in tqdm(sample_coordinates, file=sys.stdout):
            # store data
            x_d, x_s, y = {}, {}, {}
            df_native = ds.sel({"sample": sample}).to_dataframe()
            
            pass
        
        pass
    
    def prepare_data(self):
        # 1. stack data (time, lat, lon) -> (time, pixel)
        ds, samples = _stack_xarray(self.ds, pixel_vars)
        null_pixels = get_all_null_pixels_from_dataset(ds, target_var)
        ds = ds.sel(sample = ~np.isin(ds.sample, null_pixels.values))

        # 2. create the lagged_variables
        _lagged_variables = []
        for var in self.lagged_variables:
            ds, var_name = _add_lagged_feature(ds, var, lag=self.lag)
            _lagged_variables.append(var_name)
        self.lagged_variables = _lagged_variables

        # 3. get only the used data variables
        keep_cols = ["sample"] + self.dynamic_inputs + [self.target_var] + self.lagged_variables
        ds = ds[keep_cols]
                
        # 4. normalise the data based on the scaler
        # 5. create the pixel id to integer OHE
        return

    def _get_data_variables(self) -> List[str]:
        data_variables = [self.target_var] + self.lagged_variables + self.dynamic_inputs
        
    def compute_static_scaler(self, df: pd.DataFrame):
        self.scaler["static_means"] = df.mean()
        self.scaler["static_stds"] = df.std()
    
    def _apply_static_normalisation(self, df: pd.DataFrame) -> pd.DataFrame:
        df = (df - self.scaler['attribute_means']) / self.scaler["attribute_stds"]
        return df
    
    def _setup_normalization(self, ds: xr.Dataset):
        self.scaler["dynamic_stds"] = ds.std(skipna=True)
        self.scaler["dynamic_means"] = ds.mean(skipna=True)

    def normalise_data(self):
        pass

    def setup(self, stage: Optional[str] = None):
        # Assign Train/val split(s) for use in Dataloaders
        if stage == 'fit' or stage is None:
            # load in data
            # transform data (normalise with scaler)
            # transformations (log)
            self.train_data, self.val_data = random_split(mnist_full, [55000, 5000])
            self.dims = self.train_data[0][0].shape

        # Assign Test split(s) for use in Dataloaders
        if stage == 'test' or stage is None:
            self.mnist_test = MNIST(
                self.data_dir,
                train=False,
                download=True,
                transform=self.transform
            )
            self.dims = getattr(self, 'dims', self.test_data[0][0].shape)
    
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size)

    def _init_checks(self, ds: xr.Dataset) -> None:
        # check all init arguments
        assert all(np.isin(self.pixel_vars, [l for l in ds.coords])), f"Expect to find {self.pixel_vars} in {ds.coords}"
        assert target_var in ds.data_vars, f"Expect to find {target_var} in {ds.data_vars}"
        if self.lagged_variables != []:
            assert all(np.isin(self.lagged_variables, [v for v in ds.data_vars])), f"Expect to find {self.lagged_variables} in {ds.data_vars}"
        if self.dynamic_inputs != []:
            assert all(np.isin(self.dynamic_inputs, [v for v in ds.data_vars])), f"Expect to find {self.dynamic_inputs} in {ds.data_vars}"

        if (dynamic_inputs == []) & (lagged_variables == []):
            assert False, "Need to include some input features in `lagged_variables` or `dynamic_inputs`"
        
        assert ds.time.dtype == np.dtype('<M8[ns]'), "Time should be of datetime type"
            
            
d = SpatioTemporalDataset(_ds, seq_len=3, target_var="boku_VCI", lagged_variables=["boku_VCI"], dynamic_inputs=["precip"])
d.prepare_data()

In [ ]:
def encode_doys(
    doys: Union[List[str], List[int], List[datetime.datetime], str, int, datetime.datetime],
    start_doy: int = 1,
    end_doy: int = 366
) -> Tuple[List[float], List[float]]:
    doys_sin = []
    doys_cos = []
    for doy in doys:
        if isinstance(doy, str):
            doy = int(doy)
        elif isinstance(doy, datetime.datetime):
            doy = int(doy.strftime('%j'))

        if doy > 9999999:
            doy = int(datetime.datetime.strptime(
                str(doy), '%Y%m%d').strftime('%j'))
        else:
            if doy > 366 or doy < 1:
                raise ValueError(f'Invalid date "{doy}"')

        doys_sin.append(np.sin(2 * math.pi * (doy - start_doy) /
                               (end_doy - start_doy + 1)))
        doys_cos.append(np.cos(2 * math.pi * (doy - start_doy) /
                               (end_doy - start_doy + 1)))

    return doys_sin, doys_cos



# try xbatcher

In [219]:
import xbatcher

bgen = xbatcher.BatchGenerator(ds["boku_VCI"], input_dims={'time': 10})
for batch in bgen:
    pass
# batch["boku_VCI"]
# batch["boku_VCI"].to_dataframe()

In [94]:
# Dataset?